In [131]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import optuna
from prophet import Prophet

In [132]:
path = "/Users/amrtamer/Documents/Internship"
# path = "/content/drive/MyDrive/Internship"

In [133]:
data = pd.read_csv(path + r'/data/Volve/Volve_cleaned.csv')
data.head()

,Unnamed: 0,date,WELL_BORE_CODE,well_name,NPD_WELL_BORE_NAME,NPD_FIELD_CODE,NPD_FIELD_NAME,NPD_FACILITY_CODE,NPD_FACILITY_NAME,prod_hrs,...,AVG_CHOKE_UOM,whp,wht,choke_size_percentage,oil_vol,gas_vol,water_vol,FLOW_KIND,well_type,oil_rate
0,0,2014-04-22,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,24.0,...,%,107.362050,37.939251,78.935409,631.47,90439.09,0.0,production,OP,1166.46
1,1,2014-04-23,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,24.0,...,%,99.187011,60.756579,70.627109,1166.46,165720.39,0.0,production,OP,1166.46
2,2,2014-04-24,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,24.0,...,%,94.600770,63.046800,66.049151,1549.81,221707.31,0.0,production,OP,1549.81
3,3,2014-04-25,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,24.0,...,%,89.988092,64.547229,61.405386,1248.70,178063.52,0.0,production,OP,1248.70
4,4,2014-04-26,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,24.0,...,%,84.776810,65.723694,56.147906,1345.78,192602.19,0.0,production,OP,1345.78


In [134]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{7405: 'NO 15/9-F-1 C',
 7078: 'NO 15/9-F-11 H',
 7289: 'NO 15/9-F-15 D',
 5599: 'NO 15/9-F-12 H',
 5351: 'NO 15/9-F-14 H',
 5769: 'NO 15/9-F-5 AH'}

In [135]:
identification_column = ['well_name', 'date']
feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol']
target_column = ['oil_rate']

In [136]:
data = data[identification_column + feature_columns + target_column]
data.shape

(7987, 15)

In [137]:
features = data[feature_columns + target_column]
data[feature_columns + target_column] = (features - features.mean()) / features.std()

In [138]:
new_data = data.rename(columns={'date': 'ds', 'oil_rate': 'y'})

In [139]:
new_data.isna().any()

well_name                False
ds                       False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
y                        False
dtype: bool

In [140]:
new_data

,well_name,ds,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,wht,choke_size_percentage,oil_vol,gas_vol,water_vol,y
0,7405,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,-2.165276,3.625672,-0.464450,-0.494707,-1.131839,-0.102310
1,7405,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,-0.870666,3.145767,-0.066597,-0.099321,-1.131839,-0.102310
2,7405,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,-0.740723,2.881334,0.218487,0.194728,-1.131839,0.177706
3,7405,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,-0.655592,2.613101,-0.005438,-0.034494,-1.131839,-0.042238
4,7405,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,-0.588842,2.309418,0.066757,0.041865,-1.131839,0.028674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,2016-08-22,0.268405,-0.880933,-0.806036,-0.814175,1.149266,1.150153,-1.140330,-1.056587,-0.926875,-0.657907,-0.665422,-1.078364,-0.683109
7983,5769,2016-08-23,0.268405,-0.880933,-0.806036,-0.814175,1.158798,1.150153,-1.136215,-1.091664,-0.922827,-0.652538,-0.659200,-1.078659,-0.677835
7984,5769,2016-08-24,0.268405,-0.880933,-0.806036,-0.814175,1.141085,1.150153,-1.137527,-1.126432,-0.924116,-0.653795,-0.661759,-1.077945,-0.679069
7985,5769,2016-08-25,0.268405,-1.334973,-1.285316,-1.327391,1.140689,1.150153,-1.140776,-1.033676,-0.927897,-0.653363,-0.658056,-1.077909,-0.678646


In [141]:
wells = {}
for i in well_info.keys():
    well = new_data[new_data['well_name'] == i]
    wells[i] = well

In [142]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [143]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [144]:
wells_data = {}
for well_name, frame in wells.items():
    temp_data, data_test = train_test_split(frame, test_size=test_size_param1, shuffle=False)
    data_train, data_val = train_test_split(temp_data, test_size=test_size_param2, shuffle=False)
    data_train.reset_index(drop=True, inplace=True)
    data_val.reset_index(drop=True, inplace=True)
    data_test.reset_index(drop=True, inplace=True)
    wells_data[well_name] = {'train': data_train, 'val': data_val,'test': data_test}

for well_name in well_info.keys():
    well = wells_data[well_name]
    print(f"For well {well_name}, the shapes train, val, test shapes are {well['train'].shape}, {well['val'].shape}, {well['test'].shape} respectively")

For well 7405, the shapes train, val, test shapes are (256, 15), (86, 15), (86, 15) respectively
For well 7078, the shapes train, val, test shapes are (672, 15), (224, 15), (225, 15) respectively
For well 7289, the shapes train, val, test shapes are (459, 15), (153, 15), (154, 15) respectively
For well 5599, the shapes train, val, test shapes are (1698, 15), (566, 15), (567, 15) respectively
For well 5351, the shapes train, val, test shapes are (1632, 15), (544, 15), (544, 15) respectively
For well 5769, the shapes train, val, test shapes are (72, 15), (24, 15), (25, 15) respectively


In [145]:
days_window = 5
predictions_days = 14
training_gap = predictions_days + days_window

In [146]:
models = {well_name: Prophet() for well_name in well_info.keys()}
for well_name, model in models.items():
    data = wells_data[well_name]
    for column in data['train'][feature_columns].columns:
        model.add_regressor(column)
    print(data['train'].columns)
    model.fit(data['train'])

10:45:43 - cmdstanpy - INFO - Chain [1] start processing
Python(48848) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
10:45:43 - cmdstanpy - INFO - Chain [1] done processing
10:45:43 - cmdstanpy - INFO - Chain [1] start processing
Python(48849) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
10:45:43 - cmdstanpy - INFO - Chain [1] done processing
10:45:43 - cmdstanpy - INFO - Chain [1] start processing


Index(['well_name', 'ds', 'prod_hrs', 'bhp', 'bht', 'dp_tubing',
       'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'y'],
      dtype='object')
Index(['well_name', 'ds', 'prod_hrs', 'bhp', 'bht', 'dp_tubing',
       'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'y'],
      dtype='object')
Index(['well_name', 'ds', 'prod_hrs', 'bhp', 'bht', 'dp_tubing',
       'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'y'],
      dtype='object')


Python(48850) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
10:45:43 - cmdstanpy - INFO - Chain [1] done processing
10:45:43 - cmdstanpy - INFO - Chain [1] start processing
Python(48851) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Index(['well_name', 'ds', 'prod_hrs', 'bhp', 'bht', 'dp_tubing',
       'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'y'],
      dtype='object')


10:45:44 - cmdstanpy - INFO - Chain [1] done processing
10:45:44 - cmdstanpy - INFO - Chain [1] start processing
Python(48852) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Index(['well_name', 'ds', 'prod_hrs', 'bhp', 'bht', 'dp_tubing',
       'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'y'],
      dtype='object')


10:45:44 - cmdstanpy - INFO - Chain [1] done processing
10:45:44 - cmdstanpy - INFO - Chain [1] start processing
Python(48853) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
10:45:44 - cmdstanpy - INFO - Chain [1] done processing


Index(['well_name', 'ds', 'prod_hrs', 'bhp', 'bht', 'dp_tubing',
       'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'y'],
      dtype='object')


In [155]:
for well_name, model in models.items():
    data = wells_data[well_name]
    # Create a DataFrame for future dates including the validation period
    print(data['val'].loc[0, 'ds'], data['test'].iloc[-1]['ds'])
    print((pd.to_datetime(data['test'].iloc[-1]['ds']) - pd.to_datetime(data['val'].loc[0, 'ds'])).days)
    break

2015-04-21 2016-04-06
351


In [159]:
for well_name, model in models.items():
    data = wells_data[well_name]
    # Create a DataFrame for future dates including the validation period
    print(data['train'].shape, data['val'].shape, data['test'].shape)
    future = model.make_future_dataframe(periods=len(data['val']) + len(data['test']))
    for column in data['train'][feature_columns].columns: 
        future[column] = pd.concat([data['val'][column], data['test'][column]]).reset_index(drop=True)
    break
future

(256, 15) (86, 15) (86, 15)


,ds
0,2014-04-22
1,2014-04-23
2,2014-04-24
3,2014-04-25
4,2014-04-26
...,...
423,2015-10-05
424,2015-10-06
425,2015-10-07
426,2015-10-08


In [156]:
for well_name, model in models.items():
    data = wells_data[well_name]
    # Create a DataFrame for future dates including the validation period
    print(data['train'].shape, data['val'].shape, data['test'].shape)
    future = model.make_future_dataframe(periods=len(data['val']) + len(data['test']))
    for column in data['train'][feature_columns].columns: 
        future[column] = pd.concat([data['val'][column], data['test'][column]]).reset_index(drop=True)
    print(data['test']['prod_hrs'][data['test']['prod_hrs'].isna()])
    break
    # Make predictions
    forecast = model.predict(future)

    # Evaluate the model on the validation set
    val_forecast = forecast.tail(len(data['val']))

future[future.isna().any(axis=1)]

(256, 15) (86, 15) (86, 15)
Series([], Name: prod_hrs, dtype: float64)


,ds,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,wht,choke_size_percentage,oil_vol,gas_vol,water_vol
172,2014-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2014-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,2014-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,2014-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,2014-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,2015-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,2015-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,2015-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,2015-10-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences_df):
        self.sequences_df = sequences_df

    def __len__(self):
        return len(self.sequences_df)

    def __getitem__(self, idx):
        inputs = self.sequences_df.iloc[idx]['inputs']
        targets = self.sequences_df.iloc[idx]['targets']
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).squeeze()

train_dataset = TimeSeriesDataset(data_train)
val_dataset = TimeSeriesDataset(data_val)
test_dataset = TimeSeriesDataset(data_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [88]:
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

@torch.no_grad()
def evaluate(mode, model):
    loader_dict = {'train': train_loader, 'val': val_loader, 'test': test_loader}
    loader = loader_dict[mode]
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_targets = []
    test_losses = []

    for X_batch, Y_batch in loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model to get loss
        test_losses.append(loss.item())  # Collect test loss
        B, T = Y_batch.shape
        Y_batch = Y_batch.view(B*T)
        # Convert predictions and targets to NumPy arrays
        all_preds.append(logits.cpu().numpy())  # .cpu() moves tensor to CPU, .numpy() converts to NumPy array
        all_targets.append(Y_batch.cpu().numpy())

    # Concatenate all predictions and targets
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Calculate metrics
    metrics = calculate_metrics(all_targets, all_preds)
    metrics['Loss'] = np.mean(test_losses)
    print(f"Metrics on {mode} set:\n")
    print(f"Loss is {metrics['Loss']:.4f}")
    print(f"MAE is {metrics['MAE']:.4f}")
    print(f"RMSE is {metrics['RMSE']:.4f}")
    print(f"R² is {metrics['R2']:.4f}")
    print("\n")
    return metrics

# Define an objective function for Optuna
def objective(trial):
    test_iterations = 500
    
    vector_length = trial.suggest_int('vector_length', 512, 5120, step=512)
    num_layers = trial.suggest_int('num_layers', 1, 10)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)

    model = LSTM(input_length=len(feature_columns), hidden_length=vector_length, output_length=predictions_days, num_layers=num_layers, dropout=dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    train_loader_iter = iter(train_loader)
    # Training loop
    for i in range(test_iterations):
        model.train()
        if i % test_eval_interval == 0 or i == test_iterations - 1:
            losses = estimate_loss(model)
            trial.report(losses['val'], i)
            if trial.should_prune():
                raise optuna.TrialPruned()
        try:
            X_batch, Y_batch = next(train_loader_iter)
        except StopIteration:
            train_loader_iter = iter(train_loader)
            X_batch, Y_batch = next(train_loader_iter)

        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    val_metrics = evaluate('val', model)
    return val_metrics['Loss']

In [89]:
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Get the best hyperparameters
best_params = study.best_params
print('Best hyperparameters:', best_params)

# Train the final model with the best hyperparameters
vector_length = best_params['vector_length']
num_layers = best_params['num_layers']
dropout = best_params['dropout']
lr = best_params['lr']

[I 2024-06-25 13:18:31,064] A new study created in memory with name: no-name-0c6bd2b9-4e83-41c2-bca2-f35eede2a9f5
[W 2024-06-25 13:19:09,091] Trial 0 failed with parameters: {'vector_length': 3584, 'num_layers': 3, 'dropout': 0.22792055210061635, 'lr': 0.0009971981529885204} because of the following error: RuntimeError('MPS backend out of memory (MPS allocated: 16.97 GB, other allocations: 3.36 GB, max allowed: 20.40 GB). Tried to allocate 196.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).').
Traceback (most recent call last):
  File "/Users/amrtamer/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/d8/xf5yjrpn5zvd1j56c42jys2w0000gn/T/ipykernel_34653/1532449872.py", line 144, in objective
    optimizer.step()
  File "/Users/amrtamer/Library/Python/3.9/lib/python/site-packages/torch/optim/optimizer

RuntimeError: MPS backend out of memory (MPS allocated: 16.97 GB, other allocations: 3.36 GB, max allowed: 20.40 GB). Tried to allocate 196.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
train_metrics = evaluate('train', model)
val_metrics = evaluate('val', model)
test_metrics = evaluate('test', model)

Metrics on train set:

Loss is 0.0070
MAE is 0.0479
RMSE is 0.0832
R² is 0.9934


Metrics on val set:

Loss is 0.0299
MAE is 0.0885
RMSE is 0.1748
R² is 0.9712


Metrics on test set:

Loss is 0.0306
MAE is 0.0885
RMSE is 0.1748
R² is 0.9712




1. add time elapsed column
2. find a way to fill gaps
3. overfitting (add dropout)
5. deal with wht and whp
4. load model
evaulatiob=n on all e`xamples